本檔案使用資料處理後的Mel頻譜圖進行訓練與預測

為了滿足預訓練模型所需，在進入模型之前先將影像疊為三通道。

訓練結果的模型權重將存為「mels.h5'」

In [ ]:
import pandas as pd
import numpy as np
import math
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

import keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from keras_self_attention import SeqSelfAttention
from tensorflow.keras import initializers, regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D,Conv2D, MaxPooling1D, MaxPooling2D, Flatten, Dropout,Layer,Concatenate,Input,Activation
from tensorflow.keras.layers import BatchNormalization,LayerNormalization,LeakyReLU,ReLU,Add,AveragePooling1D
from tensorflow.keras.layers import GlobalAveragePooling1D, Reshape, multiply, add, GlobalMaxPooling1D, Multiply
from tensorflow.keras.layers import Masking, Bidirectional, LSTM
from tensorflow.keras.applications import EfficientNetB4
from focal_loss import SparseCategoricalFocalLoss

In [ ]:
sample_rate = 16000
data = np.load('/home/user8008//sdk/sail/audio/cup/final/train_data(1000)(normalize).npz')
data_Mel = np.load('/home/user8008/sdk/sail/audio/cup/final/MelS(train).npz')

In [ ]:
train_y = data['train_y']
train_y_bln =  data['train_y_bln']
train_MelS = data_Mel['MelS'] 
train_MelS_3 = np.stack((train_MelS, train_MelS, train_MelS), axis=3)
train_y.shape, train_y_bln.shape,train_MelS.shape

In [ ]:
MelS_x_train, MelS_x_val, y_train, y_val = train_test_split(train_MelS_3,train_y,test_size=0.2,random_state=5473,stratify=train_y_bln)

In [ ]:
def gelu(x):
    return 0.5 * x * (1.0 + K.tanh(math.sqrt(2.0 / math.pi) * (x + 0.044715 * x * x * x)))

def spatialAttention_eff(input_feature,kernel_size=7,name=""):
    cbam_feature=input_feature
    avg_pool=tf.reduce_mean(input_feature,axis=3,keepdims=True) 
    max_pool=tf.reduce_max(input_feature,axis=3,keepdims=True)
    concat=Concatenate(axis=3)([avg_pool,max_pool])
    cbam_feature=Conv2D(filters=112,kernel_size=7,strides=1,padding='same',use_bias=False,name="spatial_attention_" + str(name), dilation_rate=2)(concat)
    cbam_feature=Activation('sigmoid')(cbam_feature)
    out=Multiply()([input_feature,cbam_feature])
    return out

def residual_block_eff(x, filters, conv_num=3, activation="relu"):
    s = Conv2D(filters, 1, padding="same", dilation_rate=12)(x)  # 5
    for i in range(conv_num - 1):
        x = Conv2D(filters, 3, padding="same", dilation_rate=12)(x)
        x = ReLU()(x)
        x = Conv2D(filters, 3, padding="same", dilation_rate=12)(x)
        x = Add()([x, s])
        x = ReLU()(x)
    return MaxPooling2D(pool_size=3, strides=2,name='residual_end')(x)


model_mels = EfficientNetB4(include_top=False, weights='imagenet', input_shape=(184,184,3), pooling='max')  # avg
layer = tf.keras.models.Model(inputs=model_mels.input, outputs=model_mels.get_layer('block4c_project_conv').output)
x = layer.output
x = Dropout(0.25)(x)
x_sp = spatialAttention_eff(x,kernel_size=4,name="")
x = residual_block_eff(x_sp,8,3)
x = Flatten()(x)
x = Dropout(0.3)(x)
x = Dense(1280,kernel_regularizer=regularizers.l2(0.001))(x)
x = gelu(x)
x = Dropout(0.1)(x)
x = Dense(128,kernel_regularizer=regularizers.l2(0.001))(x)
x = gelu(x)
output = Dense(units=5, activation='softmax')(x)
model_mels = tf.keras.models.Model(inputs=model_mels.input, outputs=output)

In [ ]:
lr = 0.00003
# lr = 0.0001520754931920187
class_weight = (0.1,0.153,0.153,0.207,0.376)

model_mels.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),   #tf.keras.optimizers.RMSprop(learning_rate=lr,rho=0.9, epsilon=1e-8),  
              loss=SparseCategoricalFocalLoss(gamma=2,class_weight=class_weight) ,# ,class_weight=(0.1,0.15,0.14,0.3,0.31) SparseCategoricalFocalLoss(gamma=2) focal_loss(alpha=.25, gamma=2),sparse_categorical_crossentropy binary_crossentropy
              metrics=['accuracy'])

In [ ]:
batch_size = 64
epoch = 150

model_history = model_mels.fit([MelS_x_train],y_train,
                          validation_data = ([MelS_x_val],y_val), # callbacks=[EarlyStopping(monitor='val_loss', patience=20, mode='auto')],
                          batch_size=batch_size,
                          epochs=epoch)

In [ ]:
# model_mels.save_weights('mels.h5')